## User Instructions
This is a program which will create R-Maps. These are used to correlate a voxel to a continuous outcome measure.
As it stands, this software employs Pearson Correlation Coefficients, which imply it will be best to have a continuous outcome on a percent scale. If you would like to do otherwise, a Spearman Correlation is possible. 

The software will walk you through everything. 

_____
# Nifti Configuration

**Files are expected to follow a BIDS naming convention.**

**Files are expected to have subject ID in them which is identical to subject ID in the CSV**

**Files are expected to be in 2x2x2 resolution**
_____
# CSV configuration:
**Subject IDs expected to be in the nifti names**

**Subject IDs expected to be in a column of your target CSV labelled "subject"**

Imports

In [69]:
import pandas as pd
import numpy as np

Save Information

-Enter the directory you would like to save to

In [1]:
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/subiculum_cognition_and_age/figures/Figures/supplements/correlation_to_memory_network'

---

**Instructions**: Please fill out the `path` and `file_pattern` variables. 

The file_path is the shared base directory holding all files. ie) blah/blah/blah/BIDS

The file_pattern is the shared naming architectur in all files ie)  * / * / * subT1 * .nii

---

In [2]:
# What is the shared path to the folder/csv containing the nifti files/files paths for the neuroimaging files?
path_1 = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'

#What is the shared file architecture of your neuroimaging files after the base path?
file_pattern = '*_T*.nii'

In [3]:
#-----------------DO NOT TOUCH--------------------------------------------------------
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
df_1 = import_matrices_from_folder(path_1, file_pattern=file_pattern, subject_id_index=5)
df_1

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*_T*.nii


,datasets_116_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_1203_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_139_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_147_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_135_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_101_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_150_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_122_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_1202_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_146_vat_seed_compound_fMRI_efield_func_seed_T.nii,...,datasets_128_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_107_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_133_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_141_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_110_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_125_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_106_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_129_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_111_vat_seed_compound_fMRI_efield_func_seed_T.nii,datasets_140_vat_seed_compound_fMRI_efield_func_seed_T.nii
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. For example in 04-mwp1glanat_resampled.nii, this is " "
2) Proceeding the subject ID. For example in 04-mwp1glanat_resampled.nii, this is "-mwp1"

In [4]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id

def preprocess_names(df, string_preceding_id, string_proceeding_id, cols=True):
    """
    Preprocess the given dataframe by extracting and renaming the subject ID, 
    then transposing the dataframe.

    Parameters:
    - df: The dataframe to preprocess.
    - string_preceding_id: String preceding the subject ID.
    - string_proceeding_id: String proceeding the subject ID.

    Returns:
    - The preprocessed dataframe.
    """
    split_command_dict = {string_preceding_id: 1, string_proceeding_id: 0}
    if cols:
        df = extract_and_rename_subject_id(dataframe=df, split_command_dict=split_command_dict).transpose()
    else:
        df = extract_and_rename_subject_id(dataframe=df, split_command_dict=split_command_dict)
    df.index.name = 'subject'
    return df


In [5]:
string_preceding_id = 'datasets_'
string_proceeding_id = '_vat'

In [6]:
df_1 = preprocess_names(df_1, string_preceding_id, string_proceeding_id)
df_1

,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject,,,,,,,,,,,,,,,,,,,,,
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


handle NaNs if you'd like

In [9]:
import numpy as np
df_1[:] = np.nan_to_num(df_1.values, nan=0, posinf=30, neginf=-30)


In [8]:
df_1.index 

Index(['116', '1203', '139', '147', '135', '101', '150', '122', '1202', '146',
       '138', '134', '123', '149', '137', '118', '114', '145', '1201', '120',
       '103', '119', '148', '144', '115', '121', '102', '105', '109', '126',
       '143', '112', '131', '104', '127', '108', '113', '142', '130', '124',
       '128', '107', '133', '141', '110', '125', '106', '129', '111', '140'],
      dtype='object', name='subject')

Define the path to the CSV which has your clinical information

In [41]:
path_2 = '/Users/cu135/Dropbox (Partners HealthCare)/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
excel_sheet_name = 'master_list_proper_subjects' #Optional

In [42]:
# Import a CSV with the clinical data of interest
import pandas as pd
if os.path.basename(path_2).split('.')[1] == 'csv':
    df_2 = pd.read_csv(path_2)
else:
    df_2 = pd.read_excel(path_2, sheet_name=excel_sheet_name)
df_2

,subject,Age,Normalized Percent Cognitive Improvement,Z_Scored Percent Cognitive Improvement By Origin Group,Z-Scored Percent Cognitive Improvement,Percent Cognitive Improvement,Z-Scored Subiculum T By Origin Group,Z-Scored Subiculum Connectivity T,Subiculum Connectivity T,Amnesia Lesion T Map,...,Estimated Outcome,Cognitive Baseline,Z-Scored Cognitive Baseline,Z-Scored Cognitive Baseline (Lower is Better),Min Max Normalized Baseline,MinMaxNormBaseline Higher is Better,ROI to Alz Max,ROI to PD Max,Standardzied AD Max,Standardized PD Max
0,101,62.0,-0.392857,0.314066,0.314066,-21.428571,-1.282630,-1.282630,56.864683,0.447264,...,NaN,28,1.518764,-1.518764,0.72,0.28,12.222658,14.493929,-1.714513,-1.227368
1,102,77.0,-0.666667,0.013999,0.013999,-36.363636,-1.760917,-1.760917,52.970984,0.436157,...,NaN,22,0.465551,-0.465551,0.48,0.52,14.020048,15.257338,-1.155843,-1.022243
2,103,76.0,-1.447368,-0.841572,-0.841572,-78.947368,-0.595369,-0.595369,62.459631,0.497749,...,NaN,19,-0.061056,0.061056,0.36,0.64,15.118727,17.376384,-0.814348,-0.452865
3,104,65.0,-2.372549,-1.855477,-1.855477,-129.411765,-0.945206,-0.945206,59.611631,0.432617,...,NaN,17,-0.412127,0.412127,0.28,0.72,13.112424,15.287916,-1.437954,-1.014027
4,105,50.0,-0.192982,0.533109,0.533109,-10.526316,-1.151973,-1.151973,57.928350,0.193389,...,NaN,19,-0.061056,0.061056,0.36,0.64,15.086568,12.951426,-0.824344,-1.641831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,211,58.7,NaN,NaN,NaN,NaN,-0.415745,-0.189000,19.900000,NaN,...,GOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,152,69.4,NaN,NaN,NaN,NaN,-0.701419,-0.455000,17.900000,NaN,...,DECLINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,208,79.2,NaN,NaN,NaN,NaN,-0.929958,-0.669000,16.300000,NaN,...,DECLINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,223,71.1,NaN,NaN,NaN,NaN,-0.829972,-0.575000,17.000000,NaN,...,DECLINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Choose Specific Columns to Keep in the Second List
- Example: #df_2 = df_2.loc[:, ['Unnamed: 0', 'DBS response ratio']]

In [43]:
def set_subject_column_to_subject(df, subject_column, string_preceding_id='', string_proceeding_id='',):
    popped_column = df.pop(subject_column)
    df['subject'] = popped_column
    
    if all(df[subject_column].apply(lambda x: type(x) is str)):
        if string_proceeding_id != '':
            df[subject_column] = [name.split(string_proceeding_id)[0] for name in df[subject_column]]
        if string_preceding_id != '':
            df[subject_column] = [name.split(string_preceding_id)[1] for name in df[subject_column]]
        print('extracting subject ID')
    else:
        df[subject_column] = df[subject_column].astype(str)
    return df

In [44]:
subject_column = 'subject'

In [45]:
string_preceding_id = ''
string_proceeding_id = ''

It is expected there is a columnc called which has subject information in it. The information in this column must correspond in the dataframe above. If it does not exist, add it to your CSV before proceeding. 

Define the column below using:

subject_colum = 

In [46]:
import os
from calvin_utils.statistical_utils.voxelwise_statistical_testing import generate_r_map
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti

def reset_index_if_subject_is_index(df, subject_column):
    if subject_column in df.index.names:
        df.reset_index(inplace=True)
        # Reorder columns to make 'subject' the first column
        cols = [subject_column] + [col for col in df.columns if col != subject_column]
        df = df[cols]
    return df

def process_and_generate_maps(df_1, df_2, subject_column='subject', out_dir='', mask_path=None, method='pearson'):
    """
    Process the given dataframes, and generate maps based on the columns.

    Parameters:
    - df_1: First dataframe.
    - df_2: Second dataframe.
    - subject_column: The column name referring to the subject.
    - out_dir: The output directory to save the generated maps.
    """
    # Check if 'subject' is in the index or columns for df_1
    if subject_column in df_1.index.names:
        df_1 = reset_index_if_subject_is_index(df_1, subject_column)
    if subject_column in df_2.index.names:
        df_2 = reset_index_if_subject_is_index(df_2, subject_column)
        
    # Process subject column to string    
    df_1[subject_column] = df_1[subject_column].astype(int)
    df_2[subject_column] = df_2[subject_column].astype(int)
    df_1.fillna(0)
    # Iterate over column, avoiding the one with subject id in it
    for colname in [col for col in df_2.columns if col != subject_column]:
        print(f'Working on {colname}')
        merged_df = df_2[[colname, subject_column]].merge(df_1, on=subject_column, how='inner').set_index(subject_column)
        
        # Remove any rows with NaN values
        copy_df = merged_df.copy()
        try:
            merged_df.dropna(inplace=True)
            r_df, p_df, r_squared_df = generate_r_map(merged_df, mask_path=mask_path, method=method)

            view_and_save_nifti(p_df, os.path.join(out_dir, 'p_map', colname))
            view_and_save_nifti(r_df, os.path.join(out_dir, 'r_map', colname))
            view_and_save_nifti(r_squared_df, os.path.join(out_dir, 'r_squared_map', colname))
        except Exception as e:
            if "x and y must have length at least 2" in str(e):
                print('Caught exception: NaNs or Infs suspected in input data. Trying workaround.')
                copy_df.replace([np.inf, -np.inf], np.nan, inplace=True)
                copy_df.fillna(0, inplace=True)
                r_df, p_df, r_squared_df = generate_r_map(copy_df, mask_path=mask_path)

                view_and_save_nifti(p_df, os.path.join(out_dir, 'p_map', colname))
                view_and_save_nifti(r_df, os.path.join(out_dir, 'r_map', colname))
                view_and_save_nifti(r_squared_df, os.path.join(out_dir, 'r_squared_map', colname))
            else:
                print(f'Error: {e}')
    return merged_df


In [47]:
df_2.columns

Index(['subject', 'Age', 'Normalized Percent Cognitive Improvement',
       'Z_Scored Percent Cognitive Improvement By Origin Group',
       'Z-Scored Percent Cognitive Improvement',
       'Percent Cognitive Improvement',
       'Z-Scored Subiculum T By Origin Group ',
       'Z-Scored Subiculum Connectivity T', 'Subiculum Connectivity T',
       'Amnesia Lesion T Map', 'Memory Network T', 'Z-Scored Memory Network R',
       'Memory Network R', 'Subiculum Grey Matter', 'Subiculum White Matter',
       'Subiculum CSF', 'Subiculum Total', 'Standardized Age',
       'Standardized Percent Improvement',
       'Standardized Subiculum Connectivity',
       'Standardized Subiculum Grey Matter',
       'Standardized Subiculum White Matter', 'Standardized Subiculum CSF',
       'Standardized Subiculum Total', 'Disease', 'Cohort', 'City',
       'Inclusion Cohort', 'Age Group', 'Age_And_Disease',
       'Age Disease and Cohort', 'Subiculum Group By Z Score Sign',
       'Subiculum Group By Infl

In [48]:
df_2 = df_2[df_2['City']=='Toronto']
df_2 = df_2.loc[:, ['subject', 'Z-Scored Percent Cognitive Improvement']]
df_2

,subject,Z-Scored Percent Cognitive Improvement
0,101,0.314066
1,102,0.013999
2,103,-0.841572
3,104,-1.855477
4,105,0.533109
5,106,-0.028151
6,107,0.435498
7,108,0.158596
8,109,0.133118
9,110,-0.955451


What is the name of the column that contains your subject labels?

In [106]:
import os
mask_path = None
df_2 = set_subject_column_to_subject(df_2, subject_column=subject_column, string_preceding_id=string_preceding_id, string_proceeding_id=string_proceeding_id)
merged_df = process_and_generate_maps(df_1.copy(), df_2.copy(), subject_column=subject_column, out_dir=out_dir, mask_path=mask_path, method='spearman')

extracting subject ID
Working on 12mo_percent_change_Q1
Dataframes have been masked such that their shapes are:  (41, 225222)


100%|██████████| 225222/225222 [00:19<00:00, 11672.37it/s]


Data has been unmasked to shape:  (1, 902629)
Data has been unmasked to shape:  (1, 902629)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/p_map/12mo_percent_change_Q1
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_map/12mo_percent_change_Q1
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_squared_map/12mo_percent_change_Q1
Working on 12mo_percent_change_Q2
Dataframes have been masked such that their shapes are:  (41, 225222)


100%|██████████| 225222/225222 [00:17<00:00, 12723.21it/s]


Data has been unmasked to shape:  (1, 902629)
Data has been unmasked to shape:  (1, 902629)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/p_map/12mo_percent_change_Q2
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_map/12mo_percent_change_Q2
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_squared_map/12mo_percent_change_Q2
Working on 12mo_percent_change_Q3
Dataframes have been masked such that their shapes are:  (41, 225222)


...

Data has been unmasked to shape:  (1, 902629)
Data has been unmasked to shape:  (1, 902629)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/p_map/12mo_percent_change_Q10
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_map/12mo_percent_change_Q10
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_squared_map/12mo_percent_change_Q10
Working on 12mo_percent_change_Q11
Dataframes have been masked such that their shapes are:  (41, 225222)


100%|██████████| 225222/225222 [00:17<00:00, 13024.60it/s]


Data has been unmasked to shape:  (1, 902629)
Data has been unmasked to shape:  (1, 902629)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/p_map/12mo_percent_change_Q11
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_map/12mo_percent_change_Q11
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_squared_map/12mo_percent_change_Q11
Working on 12mo_percent_change_TOTAL11
Dataframes have been masked such that their shapes are:  (41, 225222)


100%|██████████| 225222/225222 [00:17<00:00, 13242.34it/s]


Data has been unmasked to shape:  (1, 902629)
Data has been unmasked to shape:  (1, 902629)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/p_map/12mo_percent_change_TOTAL11
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_map/12mo_percent_change_TOTAL11
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/Figures/correlation_to_clinical_outcome/voxelwise_ct/r_squared_map/12mo_percent_change_TOTAL11


Your R-Maps have all been generated. Consider adding Calvin as a collaborator if this was useful!

-- Calvin

## Optional - Perform Delta R-Map and Permute it for Significance

**Calculate the Observed Delta-R Map Between 2 Populations**

In [ ]:
from calvin_utils.statistical_utils.voxelwise_statistical_testing import generate_delta_r_map
delta_matrix = merged_df.copy()
observed_delta_r_map = generate_delta_r_map(delta_matrix, threshold_of_interest=65, column_of_interest='Age at DOS')

In [ ]:
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(observed_delta_r_map, (out_dir+'/over_vs_under_65_delta_r_map'))

## Calculate the Empiric Delta-R Map Distribution 
### Note, this permutes the label of the population without permuting the neuroimaging data.
### Therefore, we are testing if the separation of the r-maps is significantly due to the variable of interest. 

In [ ]:
from calvin_utils.statistical_utils.voxelwise_statistical_testing import permuted_patient_label_delta_r_map
from calvin_utils.file_utils.print_suppression import HiddenPrints
n_permutations = 2
column_of_interest = 'Age at DOS'
threshold_of_interest = 65
with HiddenPrints():
    p_count_df = permuted_patient_label_delta_r_map(dataframe_to_permute=merged_df, 
                                                observed_delta_r_map=observed_delta_r_map, 
                                                column_of_interest=column_of_interest, 
                                                threshold_of_interest=threshold_of_interest, 
                                                n_permutations=n_permutations)

In [ ]:
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(p_values_df, (out_dir+'/over_vs_under_65_delta_r_map_p_values_df'))

# 03 - Spatial Correlation of R-Maps

**Prepare a Second Set of Data**

Set Neuroimaging Path Information

In [16]:
# What is the shared path to the folder/csv containing the nifti files/files paths for the neuroimaging files?
path_1 = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/derivatives/third_level/vta_connectivity'

#What is the shared file architecture of your neuroimaging files after the base path?
file_pattern = '*fMRI_T.nii*'

In [20]:
#-----------------DO NOT TOUCH--------------------------------------------------------
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
df_1B = import_matrices_from_folder(path_1, file_pattern=file_pattern, subject_id_index=5)
df_1B

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/derivatives/third_level/vta_connectivity/*fMRI_T.nii*


,datasets_MDST43_seed_compound_fMRI_T.nii.gz,datasets_MDST04_seed_compound_fMRI_T.nii.gz,datasets_MDST12_seed_compound_fMRI_T.nii.gz,datasets_MDST01_seed_compound_fMRI_T.nii.gz,datasets_MDST28_seed_compound_fMRI_T.nii.gz,datasets_MDST17_seed_compound_fMRI_T.nii.gz,datasets_MDST18_seed_compound_fMRI_T.nii.gz,datasets_MDST31_seed_compound_fMRI_T.nii.gz,datasets_MDST27_seed_compound_fMRI_T.nii.gz,datasets_MDST34_seed_compound_fMRI_T.nii.gz,...,datasets_MDST03_seed_compound_fMRI_T.nii.gz,datasets_MDST15_seed_compound_fMRI_T.nii.gz,datasets_MDST37_seed_compound_fMRI_T.nii.gz,datasets_MDST21_seed_compound_fMRI_T.nii.gz,datasets_MDST24_seed_compound_fMRI_T.nii.gz,datasets_MDST02_seed_compound_fMRI_T.nii.gz,datasets_MDST14_seed_compound_fMRI_T.nii.gz,datasets_MDST07_seed_compound_fMRI_T.nii.gz,datasets_MDST38_seed_compound_fMRI_T.nii.gz,datasets_MDST11_seed_compound_fMRI_T.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Clean subject names

In [21]:
string_preceding_id = 'datasets_MDST'
string_proceeding_id = '_seed'

In [22]:
df_1B = preprocess_names(df_1B, string_preceding_id, string_proceeding_id)
df_1B

,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject,,,,,,,,,,,,,,,,,,,,,
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Set Clinical CSV Information

In [27]:
path_2 = '/Users/cu135/Dropbox (Partners HealthCare)/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
excel_sheet_name = 'master_list_proper_subjects' #Optional

In [28]:
# Import a CSV with the clinical data of interest
import pandas as pd
if os.path.basename(path_2).split('.')[1] == 'csv':
    df_2B = pd.read_csv(path_2)
else:
    df_2B = pd.read_excel(path_2, sheet_name=excel_sheet_name)
df_2B = df_2B[df_2B['City']=='Wurzburg']

In [29]:
df_2B

,subject,Age,Normalized Percent Cognitive Improvement,Z_Scored Percent Cognitive Improvement By Origin Group,Z-Scored Percent Cognitive Improvement,Percent Cognitive Improvement,Z-Scored Subiculum T By Origin Group,Z-Scored Subiculum Connectivity T,Subiculum Connectivity T,Amnesia Lesion T Map,...,Estimated Outcome,Cognitive Baseline,Z-Scored Cognitive Baseline,Z-Scored Cognitive Baseline (Lower is Better),Min Max Normalized Baseline,MinMaxNormBaseline Higher is Better,ROI to Alz Max,ROI to PD Max,Standardzied AD Max,Standardized PD Max
46,1,57.0,-2.609929,-1.372562,-1.372562,-5.673759,1.080695,1.080695,30.376565,-0.113151,...,NaN,141,-0.115295,-0.115295,0.625,0.625,22.020300,20.467840,1.056258,0.508516
47,2,50.0,0.992806,1.331414,1.331414,2.158273,-0.930548,-0.930548,16.295870,-0.502484,...,NaN,139,-0.935174,-0.935174,0.375,0.375,11.487188,4.942970,-0.759238,-1.629565
48,3,62.0,-0.638889,0.106772,0.106772,-1.388889,1.155469,1.155469,30.900051,-0.398033,...,NaN,144,1.114522,1.114522,1.000,1.000,23.013479,22.145924,1.227443,0.739621
49,4,50.0,-0.985714,-0.153533,-0.153533,-2.142857,-0.228971,-0.228971,21.207602,-0.426115,...,NaN,140,-0.525235,-0.525235,0.500,0.500,12.198485,18.933435,-0.636638,0.297198
50,6,60.0,-0.323944,0.343149,0.343149,-0.704225,0.109572,0.109572,23.577739,-0.454075,...,NaN,142,0.294644,0.294644,0.750,0.750,17.634088,18.128314,0.300247,0.186317
51,7,73.0,-0.326241,0.341424,0.341424,-0.709220,1.977842,1.977842,36.657479,-0.177886,...,NaN,141,-0.115295,-0.115295,0.625,0.625,24.162033,27.503198,1.425409,1.477423
52,9,64.0,-0.985714,-0.153533,-0.153533,-2.142857,-0.407778,-0.407778,19.955774,-0.494405,...,NaN,140,-0.525235,-0.525235,0.500,0.500,10.782803,14.964053,-0.880646,-0.249464
53,11,62.0,-0.319444,0.346526,0.346526,-0.694444,-1.093332,-1.093332,15.156220,-0.507962,...,NaN,144,1.114522,1.114522,1.000,1.000,9.653427,12.002916,-1.075306,-0.657271
54,12,54.0,0.000000,0.586280,0.586280,0.000000,0.788134,0.788134,28.328345,-0.220427,...,NaN,136,-2.164991,-2.164991,0.000,0.000,21.521001,21.243697,0.970198,0.615367
55,14,49.0,0.321678,0.827710,0.827710,0.699301,-0.455880,-0.455880,19.619016,-0.440579,...,NaN,143,0.704583,0.704583,0.875,0.875,10.881447,15.224677,-0.863644,-0.213571


In [31]:
df_2B = df_2B.loc[:, ['subject', 'Z-Scored Percent Cognitive Improvement']]

In [32]:
subject_column = 'subject'
string_preceding_id = ''
string_proceeding_id = ''

Process df_2 subject names

In [33]:
df_2B = set_subject_column_to_subject(df_2B, subject_column=subject_column, string_preceding_id=string_preceding_id, string_proceeding_id=string_proceeding_id)
df_2B

,Z-Scored Percent Cognitive Improvement,subject
46,-1.372562,1
47,1.331414,2
48,0.106772,3
49,-0.153533,4
50,0.343149,6
51,0.341424,7
52,-0.153533,9
53,0.346526,11
54,0.586280,12
55,0.827710,14


**Run the Spatial Correlatio Function**

In [49]:
import numpy as np
import scipy.stats
from tqdm import tqdm
import sys
from contextlib import contextmanager
from calvin_utils.statistical_utils.voxelwise_statistical_testing import generate_r_map

# # Define a dummy tqdm function
# def dummy_tqdm(*args, **kwargs):
#     if 'iterable' in kwargs:
#         return kwargs['iterable']
#     return args[0] if args else range(0)

# @contextmanager
# def suppress_print():
#     # Save the original tqdm and stdout
#     original_tqdm = tqdm
#     original_stdout = sys.stdout

#     # Replace tqdm with the dummy function and stdout with a null device
#     tqdm = dummy_tqdm
#     sys.stdout = open(os.devnull, 'w')

#     try:
#         yield
#     finally:
#         # Restore the original tqdm and stdout
#         tqdm = original_tqdm
#         sys.stdout = original_stdout
        
@contextmanager
def suppress_print():
    # Open a null device
    with open(os.devnull, 'w') as devnull:
        # Save the current stdout
        old_stdout = sys.stdout
        # Redirect the current stdout to the null device
        sys.stdout = devnull
        try:
            # Yield back to the calling function
            yield
        finally:
            # Restore the original stdout
            sys.stdout = old_stdout

class SpatialCorrelRMaps:
    def __init__(self, df_1, df_2, df_1B, df_2B, subject_column='subject', out_dir=None, mask_path=None, method='pearson'):
        self.df_1 = df_1
        self.df_2 = df_2
        self.df_1B = df_1B
        self.df_2B = df_2B
        self.subject_column = subject_column
        self.out_dir = out_dir
        self.mask_path = mask_path
        self.method = method


    def spatial_correlation(self, r_map1, r_map2):
        r_map1 = np.nan_to_num(r_map1.to_numpy().flatten(), neginf=1, posinf=0, nan=0)
        r_map2 = np.nan_to_num(r_map2.to_numpy().flatten(), neginf=1, posinf=0, nan=0)
        correlation_coefficient, _ = scipy.stats.pearsonr(r_map1, r_map2)
        return correlation_coefficient

    def permute_subjects(self, df):
        df_permuted = df.copy()
        df_permuted[self.subject_column] = np.random.permutation(df[self.subject_column])
        return df_permuted
    
    def r_map(self, df_1, df_2):
        """
        Process the given dataframes, and generate maps based on the columns.

        Parameters:
        - df_1: First dataframe.
        - df_2: Second dataframe.
        - subject_column: The column name referring to the subject.
        - out_dir: The output directory to save the generated maps.
        """
        # Check if 'subject' is in the index or columns for df_1
        if subject_column in df_1.index.names:
            df_1 = reset_index_if_subject_is_index(df_1, subject_column)
        if subject_column in df_2.index.names:
            df_2 = reset_index_if_subject_is_index(df_2, subject_column)
            
        # Process subject column to string    
        df_1[subject_column] = df_1[subject_column].astype(str)
        df_2[subject_column] = df_2[subject_column].astype(str)
        
        # Iterate over column, avoiding the one with subject id in it
        for colname in [col for col in df_2.columns if col != subject_column]:
            merged_df = df_2[[colname, subject_column]].merge(df_1, on=subject_column, how='inner').set_index(subject_column)
            
            # Remove any rows with NaN values
            copy_df = merged_df.copy()
            try:
                merged_df.dropna(inplace=True)
                with suppress_print():
                    r_df, _, _ = generate_r_map(merged_df, mask_path=self.mask_path, method=self.method, tqdm_on=False)
            except Exception as e:
                if "x and y must have length at least 2" in str(e):
                    print('Caught exception: NaNs or Infs suspected in input data. Trying workaround.')
                    copy_df.replace([np.inf, -np.inf], np.nan, inplace=True)
                    copy_df.fillna(0, inplace=True)
                    r_df, _, _ = generate_r_map(copy_df, mask_path=self.mask_path, tqdm_on=False)
                else:
                    print(f'Error {e}')
        return r_df

    def observed_distribution(self):
        r_map1 = self.r_map(self.df_1, self.df_2)
        r_map2 = self.r_map(self.df_1B, self.df_2B)
        observed_corr = self.spatial_correlation(r_map1, r_map2)
        return observed_corr

    def empiric_distribution(self, n_permutations):
        empiric_corrs = []
        for _ in tqdm(range(n_permutations)):
            permuted_df_2 = self.permute_subjects(self.df_2)
            permuted_df_2B = self.permute_subjects(self.df_2B)
            r_map1 = self.r_map(self.df_1, permuted_df_2)
            r_map2 = self.r_map(self.df_1B, permuted_df_2B)
            corr = self.spatial_correlation(r_map1, r_map2)
            empiric_corrs.append(corr)
        return empiric_corrs
    
    def p_value(self, observed_corr, empiric_corrs):
        """
        Calculate the p-value for the observed spatial correlation.

        Parameters:
            observed_corr (float): The observed spatial correlation coefficient.
            empiric_corrs (list): A list of spatial correlation coefficients from permuted data.

        Returns:
            float: The p-value representing the statistical significance of the observed correlation.
        """
        # Count how many empiric correlations are greater than or equal to the observed correlation
        count_greater = sum(emp_corr >= observed_corr for emp_corr in empiric_corrs)

        # Calculate the p-value (proportion of empiric correlations greater than or equal to the observed)
        p_val = count_greater / len(empiric_corrs)
        return p_val

    def run(self, n_permutations=100):
        """
        Execute the entire process of calculating spatial correlations and p-value.

        Parameters:
            n_permutations (int): Number of permutations for the empirical distribution calculation.

        Returns:
            tuple: A tuple containing the observed correlation, the empirical correlations, and the p-value.
        """
        # Calculate the observed spatial correlation
        observed_corr = self.observed_distribution()

        # Calculate the empirical distribution of spatial correlations from permuted data
        empiric_corrs = self.empiric_distribution(n_permutations)

        # Calculate the p-value for the observed spatial correlation
        p_val = self.p_value(observed_corr, empiric_corrs)
        print(f"Observed Correlation: {observed_corr}")
        print(f"P-Value: {p_val}")
        return observed_corr, empiric_corrs, p_val


    # Placeholder for any additional methods, such as p-value calculation or utility functions


Run It/

In [50]:
# Assuming df_1, df_2, df_1B, df_2B are predefined DataFrames
# Initialize the SpatialCorrelRMaps instance
spatial_correl = SpatialCorrelRMaps(df_1, df_2, df_1B, df_2B, 
                                    subject_column='subject', 
                                    out_dir=out_dir, 
                                    mask_path=None, 
                                    method='pearson')
obsv, empir, pval = spatial_correl.run()

100%|██████████| 100/100 [1:24:41<00:00, 50.81s/it]

Observed Correlation: -0.25493455663100756
P-Value: 0.99


In [51]:
spatial_correl.p_value

<bound method SpatialCorrelRMaps.p_value of <__main__.SpatialCorrelRMaps object at 0x18fe88160>>

Enjoy

--Calvin